In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")
from tqdm.notebook import tqdm
import time
from IPython.display import clear_output

import ee, geemap

In [2]:
%load_ext autoreload
%autoreload 2
import sys  
sys.path.insert(0, '../lib/')

In [3]:
%reload_ext autoreload
import sand_classification as sc
import quality_mosaic as qm

# Part I: Get the locations from Google Drive

In [4]:
sheet_id = '13nF_pJ02Bd70cDJamuKbvZIkIdJ-kOI4O3Cx9K7Wzos'
# sheet_id = '1XqOtPkiE_Q0dfGSoyxrH730RkwrTczcRbDeJJpqRByQ'
sheet_name = 'florida-fwc'
drive_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
drive_url

'https://docs.google.com/spreadsheets/d/13nF_pJ02Bd70cDJamuKbvZIkIdJ-kOI4O3Cx9K7Wzos/gviz/tq?tqx=out:csv&sheet=florida-fwc'

In [5]:
output_file_path = '../data/labels/gt-bands.xlsx'

In [6]:
#Read in a google sheets with the ground truth data from string drive_loc
df = pd.read_csv(drive_url)
df

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes,Unnamed: 9,...,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27
0,fwc-0,29.730603,-81.236916,sand,2021-07-05,coastal,https://gis.myfwc.com/Data/KMZ_files/ESI%20-%2...,USA,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,fwc-1,29.903629,-81.277650,sand,2019-11-15,coastal,https://gis.myfwc.com/Data/KMZ_files/ESI%20-%2...,USA,"Cape Francis, FL",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,fwc-2,30.366517,-86.314372,sand,2019-11-15,coastal,https://gis.myfwc.com/Data/KMZ_files/ESI%20-%2...,USA,"Miramar Beach, FL (fine and medium sand)",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,fwc-3,29.574251,-81.172188,sand,2019-11-16,coastal,https://gis.myfwc.com/Data/KMZ_files/ESI%20-%2...,USA,"Beverly Beach, FL (coarse sand)",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,fwc-4,30.363937,-86.301247,sand,2021-01-04,coastal,https://gis.myfwc.com/Data/KMZ_files/ESI%20-%2...,USA,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,fwc-5,30.383364,-86.850119,sand,2021-01-04,coastal,https://gis.myfwc.com/Data/KMZ_files/ESI%20-%2...,USA,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,fwc-6,29.869759,-85.404242,sand,2020-05-15,coastal,https://gis.myfwc.com/Data/KMZ_files/ESI%20-%2...,USA,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#look for any NaN values in the column 'date' or 'Longitude' or 'Latitude' in df_merged and drop them
df = df.dropna(subset=['Date', 'Longitude', 'Latitude']).reset_index(drop=True)
#drop any columns that have 'Unnamed:' in the column name
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes
0,fwc-0,29.730603,-81.236916,sand,2021-07-05,coastal,https://gis.myfwc.com/Data/KMZ_files/ESI%20-%2...,USA,NaN
1,fwc-1,29.903629,-81.277650,sand,2019-11-15,coastal,https://gis.myfwc.com/Data/KMZ_files/ESI%20-%2...,USA,"Cape Francis, FL"
2,fwc-2,30.366517,-86.314372,sand,2019-11-15,coastal,https://gis.myfwc.com/Data/KMZ_files/ESI%20-%2...,USA,"Miramar Beach, FL (fine and medium sand)"
3,fwc-3,29.574251,-81.172188,sand,2019-11-16,coastal,https://gis.myfwc.com/Data/KMZ_files/ESI%20-%2...,USA,"Beverly Beach, FL (coarse sand)"
4,fwc-4,30.363937,-86.301247,sand,2021-01-04,coastal,https://gis.myfwc.com/Data/KMZ_files/ESI%20-%2...,USA,NaN
5,fwc-5,30.383364,-86.850119,sand,2021-01-04,coastal,https://gis.myfwc.com/Data/KMZ_files/ESI%20-%2...,USA,NaN
6,fwc-6,29.869759,-85.404242,sand,2020-05-15,coastal,https://gis.myfwc.com/Data/KMZ_files/ESI%20-%2...,USA,NaN


In [8]:
assert(len(df[df['Date'].isna() | df['Longitude'].isna() | df['Latitude'].isna()]) == 0)

# Part II: Select subset for training

Here we iterate over the df and iteratively pick and adjust the locations we want to keep

In [9]:
df.columns

Index(['ID', 'Latitude', 'Longitude', 'Class', 'Date', 'Site', 'Source',
       'Country', 'Notes'],
      dtype='object')

In [10]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()


In [11]:
output = df.copy()
output

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes
0,fwc-0,29.730603,-81.236916,sand,2021-07-05,coastal,https://gis.myfwc.com/Data/KMZ_files/ESI%20-%2...,USA,NaN
1,fwc-1,29.903629,-81.277650,sand,2019-11-15,coastal,https://gis.myfwc.com/Data/KMZ_files/ESI%20-%2...,USA,"Cape Francis, FL"
2,fwc-2,30.366517,-86.314372,sand,2019-11-15,coastal,https://gis.myfwc.com/Data/KMZ_files/ESI%20-%2...,USA,"Miramar Beach, FL (fine and medium sand)"
3,fwc-3,29.574251,-81.172188,sand,2019-11-16,coastal,https://gis.myfwc.com/Data/KMZ_files/ESI%20-%2...,USA,"Beverly Beach, FL (coarse sand)"
4,fwc-4,30.363937,-86.301247,sand,2021-01-04,coastal,https://gis.myfwc.com/Data/KMZ_files/ESI%20-%2...,USA,NaN
5,fwc-5,30.383364,-86.850119,sand,2021-01-04,coastal,https://gis.myfwc.com/Data/KMZ_files/ESI%20-%2...,USA,NaN
6,fwc-6,29.869759,-85.404242,sand,2020-05-15,coastal,https://gis.myfwc.com/Data/KMZ_files/ESI%20-%2...,USA,NaN


In [12]:
%reload_ext autoreload
import sand_classification as sc
import quality_mosaic as qm
import file_management as fm

In [15]:
INDEX = 4

In [14]:
sampling_buffer_m = 5
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Manual Loop

Keep running the following 2 cells until you're done with the manual sampling.


In [83]:
#Uncomment to change the date of the image
output['Date'].iloc[INDEX] = '2021-07-11'
# output['ID'].iloc[INDEX] = 'ns-49'

In [21]:
s1_s2, sample = qm.get_s1s2_data(output, Map, INDEX, display_smap=False, mosaic_method='median', sampling_buffer_m=sampling_buffer_m, 
                                 max_search_window_months=3, median_samples=5,roi_buffer_m=5000, obia=True)

Index:  6  ID:  fwc-6 Class:  sand  Site:  coastal
Search window from 2020-02-15 to 2020-08-15


## Use this immediately after the map is refreshed to either select or discard the sample

INDEX will auto update on calling this function.
It will also overwrite just the sheet if the same exists, but will preserve the other sheets in the book

In [22]:
output, INDEX = qm.get_training_sample(output, s1_s2, sample, Map, INDEX, display_clusters=False, obia=True)
# fm.write_to_excel(output, output_file_path, sheet_name)
with pd.ExcelWriter(output_file_path,engine="openpyxl",mode="a",if_sheet_exists="replace") as writer:
    output.to_excel(writer,sheet_name=sheet_name,index=False)
output.iloc[INDEX-1]

Original marker accepted
Kept Observation


ID                                                              fwc-6
Latitude                                                    29.869759
Longitude                                                  -85.404242
Class                                                            sand
Date                                                       2020-05-15
Site                                                          coastal
Source              https://gis.myfwc.com/Data/KMZ_files/ESI%20-%2...
Country                                                           USA
Notes                                                             NaN
B2_mean                                                   4849.619629
B3_mean                                                   5355.782715
B4_mean                                                   5805.141113
B8_mean                                                   6504.173828
B8A_mean                                                  6249.130371
B11_mean            

## Final Output

In [23]:
output

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes,B2_mean,...,B11_mean,B12_mean,VV_mean,VH_mean,mTGSI_mean,BSI_mean,NDWI_mean,keep,location_tweaked,class_code
0,fwc-0,29.730603,-81.236916,sand,2021-07-05,coastal,https://gis.myfwc.com/Data/KMZ_files/ESI%20-%2...,USA,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
1,fwc-1,29.903629,-81.277650,sand,2019-11-15,coastal,https://gis.myfwc.com/Data/KMZ_files/ESI%20-%2...,USA,"Cape Francis, FL",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
2,fwc-2,30.366517,-86.314372,sand,2019-11-15,coastal,https://gis.myfwc.com/Data/KMZ_files/ESI%20-%2...,USA,"Miramar Beach, FL (fine and medium sand)",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
3,fwc-3,29.574251,-81.172188,sand,2019-11-16,coastal,https://gis.myfwc.com/Data/KMZ_files/ESI%20-%2...,USA,"Beverly Beach, FL (coarse sand)",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
4,fwc-4,30.363937,-86.301247,sand,2021-01-04,coastal,https://gis.myfwc.com/Data/KMZ_files/ESI%20-%2...,USA,NaN,5485.152344,...,5362.927734,5049.818848,-19.338247,-26.190538,-0.033975,-0.028622,-0.057828,True,False,1
5,fwc-5,30.383364,-86.850119,sand,2021-01-04,coastal,https://gis.myfwc.com/Data/KMZ_files/ESI%20-%2...,USA,NaN,4607.806641,...,6673.612793,6223.596680,-18.753519,-27.739109,0.039849,0.068058,-0.081419,True,False,1
6,fwc-6,29.869759,-85.404242,sand,2020-05-15,coastal,https://gis.myfwc.com/Data/KMZ_files/ESI%20-%2...,USA,NaN,4849.619629,...,6349.836914,6176.788086,-17.590448,-26.417429,0.021239,0.033386,-0.096750,True,False,1
